<a href="https://colab.research.google.com/github/AishaEvering/PyTorch_Exercises/blob/main/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [6]:
%%writefile get_data.py
"""
Downloads data given a path, upzips and saves it to disk.
"""
import os
import zipfile
import requests
import argparse

parser = argparse.ArgumentParser(prog="GetData",
                                 description="Downloads data given a path, upzips and saves it to disk.")
parser.add_argument('--data_path', dest="data_path", help="Path where the data will be saved.")
parser.add_argument('--data_name', dest="data_name", help="Name of the data.")
parser.add_argument('--src_url', dest="src_url", help="URL to source data.")

args = parser.parse_args()

DEFAULT_NAME = "pizza_steak_sushi"
DEFAULT_SRC_URL = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"

# Setup path to data folder
data_path = args.data_path if args.data_path else "data/"
image_path = os.path.join(data_path, args.data_name if args.data_name else DEFAULT_NAME)

# If the image folder doesn't exist, download it and prepare it...
if os.path.isdir(image_path):
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    os.makedirs(image_path, exist_ok=True)

zip_full_path = os.path.join(data_path, f"{DEFAULT_NAME}.zip")

# Download pizza, steak, sushi data
with open(zip_full_path, "wb") as f:
    request = requests.get(args.src_url if args.src_url else DEFAULT_SRC_URL)
    print(f"Downloading {DEFAULT_NAME} data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(zip_full_path, "r") as zip_ref:
    print(f"Unzipping {DEFAULT_NAME} data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(zip_full_path)

Overwriting get_data.py


In [7]:
# Example running of get_data.py
!python get_data.py

data/pizza_steak_sushi directory exists.
Unzipping pizza_steak_sushi data...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [8]:
import os
import requests

def fetch_url(entry):
  path, uri = entry

  if not os.path.exists(path):
    r = requests.get(uri, stream=True)

    if r.status_code == 200:
      with open(path, 'wb') as f:
        for chunk in r:
          f.write(chunk)
  return path

In [24]:
# download required python scripts
# I previously made this in Chapter 5

from multiprocessing.pool import ThreadPool

urls = [
    ("data_setup.py","https://raw.githubusercontent.com/AishaEvering/PyTorch_Exercises/main/assests/data_setup.py"),
    ("engine.py","https://raw.githubusercontent.com/AishaEvering/PyTorch_Exercises/main/assests/engine.py"),
    ("model_builder.py","https://raw.githubusercontent.com/AishaEvering/PyTorch_Exercises/main/assests/model_builder.py"),
    ("utils.py","https://raw.githubusercontent.com/AishaEvering/PyTorch_Exercises/main/assests/utils.py")
]

results = ThreadPool(8).imap_unordered(fetch_url, urls)

for path in results:
  print(path)

data_setup.py
engine.py
model_builder.py
utils.py


In [31]:
%%writefile train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""
import os
import torch
from torchvision import transforms
import data_setup, engine, model_builder, utils
import argparse
from functools import wraps
from time import perf_counter

# create timer decorator
def get_time(func):
  """
  Times training function
  """

  @wraps(func)
  def wrapper(*args, **kwargs):
    start_time: float = perf_counter()
    func(*args, **kwargs)
    end_time: float = perf_counter()
    print(f"[INFO] Total execution time: {end_time-start_time:.3f} seconds")

  return wrapper

# create arguments
parser = argparse.ArgumentParser(prog="Train", description="Trains a model")

parser.add_argument('--num_epochs', dest="num_epochs", default=5, help="Number of epochs.")
parser.add_argument('--batch_size', dest="batch_size", default=32, help="Number of batches.")
parser.add_argument('--hidden_units', dest="hidden_units", default=10, help="Number of hidden units.")
parser.add_argument('--learning_rate', dest="learning_rate", default=0.001, help="Optimizer learning rate.")
parser.add_argument('--train_dir', dest="train_dir", default="data/pizza_steak_sushi/train", help="Training data directory.")
parser.add_argument('--test_dir', dest="test_dir", default="data/pizza_steak_sushi/test", help="Testing data directory.")
parser.add_argument('--model_save_as', dest="model_save_as", default="05_going_modular_script_mode_tinyvgg_model.pth",
                    help="Name the model will be saved as.")

args = parser.parse_args()

@get_time
def train(args: argparse.Namespace) -> None:
  # Setup directories
  train_dir = args.train_dir
  test_dir = args.test_dir

  # Setup device agnostic code
  device = "cuda" if torch.cuda.is_available() else "cpu"

  data_transform = transforms.Compose([transforms.Resize((64, 64)),
                                      transforms.ToTensor()])

  # Create DataLoaders's and get class_names
  train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                  test_dir=test_dir,
                                                                                  transform=data_transform,
                                                                                  batch_size=int(args.batch_size))

  print(f'Input Shape: {3}\nHidden Units: {args.hidden_units}\nOutput Shape:{len(class_names)}')
  # Create model
  model = model_builder.TinyVGG(input_shape=3, hidden_units=int(args.hidden_units), output_shape=len(class_names)).to(device)

  # Setup loss and optimizer
  loss_fn = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=float(args.learning_rate))

  # Start training with help from engine.py
  engine.train(model=model,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              loss_fn=loss_fn,
              optimizer=optimizer,
              epochs=int(args.num_epochs),
              device=device)

  # Save the model to file
  utils.save_model(model=model,
              target_dir="models",
              model_name=args.model_save_as)


# call train
train(args)

Overwriting train.py


In [32]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

Input Shape: 3
Hidden Units: 128
Output Shape:3
  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1032 | train_acc: 0.3500 | test_loss: 1.0938 | test_acc: 0.3333
 20% 1/5 [00:01<00:05,  1.44s/it]Epoch: 2 | train_loss: 1.0961 | train_acc: 0.3485 | test_loss: 1.0938 | test_acc: 0.3733
 40% 2/5 [00:02<00:03,  1.23s/it]Epoch: 3 | train_loss: 1.0812 | train_acc: 0.4726 | test_loss: 1.0731 | test_acc: 0.3867
 60% 3/5 [00:03<00:02,  1.15s/it]Epoch: 4 | train_loss: 1.0528 | train_acc: 0.5344 | test_loss: 1.0439 | test_acc: 0.4533
 80% 4/5 [00:04<00:01,  1.21s/it]Epoch: 5 | train_loss: 1.0094 | train_acc: 0.5697 | test_loss: 1.0187 | test_acc: 0.4400
100% 5/5 [00:06<00:00,  1.30s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth
[INFO] Total execution time: 6.628 seconds


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [61]:
%%writefile predict.py

"""
Loads a model and makes a prediction on the given image.
"""
import torch
import torchvision
import argparse
from torch import nn
from torchvision import transforms
from typing import Tuple
import model_builder


# create arguments
parser = argparse.ArgumentParser(prog="Train", description="Trains a model")
parser.add_argument('--image', dest="image", help="Image used in prediction.")
parser.add_argument('--model_path', dest="model_path", default="models/05_going_modular_script_mode_tinyvgg_model.pth", help="Path to trained model.")
args = parser.parse_args()

def transform(img: torch.Tensor, expected_size: Tuple = (64, 64)) -> torch.Tensor:
  # create transform pipeline
  img_transformer = transforms.Compose([transforms.Resize(size=expected_size)])

  # resize the image
  return img_transformer(img)

def get_predicted_class(img: torch.Tensor, model: torch.nn.Module, device: str) -> Tuple[str, float]:
  model.eval()
  with torch.inference_mode():
    image_preds = model(img.to(device))
    pred_probabilities = torch.softmax(image_preds, dim=1)
    predicted_label = torch.argmax(pred_probabilities, dim=1).cpu()

  class_labels = ['pizza', 'steak', 'sushi']

  return class_labels[predicted_label], pred_probabilities.max()

def predict_image(image_path: str, model: torch.nn.Module, device: str, expected_size: tuple = (64, 64)) -> None:
  '''
    Makes a prediction on a target image with a trained model and plots the image and prediciton.
  '''
  # read image
  img = torchvision.io.read_image(image_path)

  # update the type
  img = img.type(torch.float32)

  # scale the image
  img = img / 255

  # resize the image
  img = transform(img)

  # add batch size dimension
  img = img.unsqueeze(0)

  # make sure model is on target device
  model.to(device)

  # predict
  label, pred_prob = get_predicted_class(img, model, device)

  print(f'[INFO] Pred: {label} | Prob: {(pred_prob * 100):.2f}%')

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# load model
loaded_model = model_builder.TinyVGG(input_shape=3, hidden_units=128, output_shape=3).to(device)
loaded_model.load_state_dict(torch.load(args.model_path))
predict_image(args.image, loaded_model, device)

Overwriting predict.py


In [64]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

[INFO] Pred: pizza | Prob: 42.59%
